# Metadata

**L1 Taxonomy** - Software Architecture & Design

**L2 Taxonomy** - Observer Pattern

**Subtopic** - Implementing a Subject class with attach, detach, notify and an Observer interface with update method

**Use Case** - Implement a Subject class in Python with methods to attach, detach, and notify observers. Also, create an Observer interface with an update method. The Subject class should maintain a list of observers and provide methods to add (attach) an observer, remove (detach) an observer, and notify all attached observers of changes. The Observer interface should define an update method that observers must implement. Use this pattern to model a scenario where a data object (the Subject) is updated and multiple other objects (the Observers) need to be informed of this change.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

**Problem Description**
You are building a flexible instrumentation framework around core algorithms.  A generic **Subject/Observer** system must allow any number of “algorithm executors” (subjects) to publish fine-grained events as they run, and observers to attach, detach, filter, and even fail without breaking the underlying logic.  You’ll demonstrate this by instrumenting three executors—MergeSort, QuickSort, and BFS—and layering on observers that log, filter, one-shot unsubscribe, and inject errors.

**Input Format and Constraints**

* No direct stdin/stdout.
* All interaction happens via method calls on instances of executor classes (subjects).
* Algorithms operate on built-in Python data structures: lists of integers or adjacency maps.
* Observers implement an `update(event: str, data: Any) → None` method.

**Expected Output Format**

* Observers may print to console, collect events, or modify external state—but the executors themselves return only their algorithmic results:

  * `MergeSortExecutor.sort(arr) → List[int]`
  * `QuickSortExecutor.sort(arr) → List[int]`
  * `BFSExecutor.bfs(graph, start) → Set[Any]`



# Requirements


**Requirements**

1. **Core Observer Pattern**

   * Define an abstract `Observer` interface with `update(event, data)`.
   * Define a `Subject` base class that maintains a list of observers and supports `attach()`, `detach()`, and `notify(event, data)`.

2. **Algorithm Executors**

   * **MergeSortExecutor** emits events:

     * `'start'` with the original list,
     * `'compare'` on each element comparison,
     * `'merge'` for each merged sublist,
     * `'end'` with the fully sorted list.
   * **QuickSortExecutor** emits:

     * `'start'`,
     * `'pivot'` when choosing a pivot,
     * `'compare'` for each partition comparison,
     * `'swap'` whenever elements are swapped,
     * `'end'`.
   * **BFSExecutor** emits:

     * `'bfs_start'`,
     * `'visit'` on each dequeued node,
     * `'enqueue'` for each neighbor queued,
     * `'error'` if the start node is missing,
     * `'bfs_end'` with the visited set.

3. **Observer Behaviors**

   * **PrintObserver**: logs every event.
   * **FilterObserver**: only logs a configurable subset of event types.
   * **OneShotObserver**: unsubscribes itself after seeing a specific trigger event.
   * **ErrorObserver**: throws a `RuntimeError` on a configured event to verify that failures are caught and don’t stop notification to others.

4. **Algorithmic Edge Cases**

   * MergeSort on empty lists, singletons, and all-duplicates.
   * QuickSort on reversed lists, identical elements.
   * BFS on missing start node and graphs with cycles and dangling keys.

5. **Robustness**

   * Observers’ exceptions must be caught and logged by the subject without halting the executor.
   * Detaching during notification (e.g. one-shot) must not break the current loop.
   * Swallow invalid detach calls (no error if observer was never attached).

6. **Performance**

   * Core sort and BFS runtimes must remain their usual complexities:

     * MergeSort and QuickSort: $O(n\log n)$,
     * BFS: $O(V + E)$.
   * Observer overhead is $O(k)$ per event (k = number of observers).

**Function Signatures**

```python
class Observer(Protocol):
    def update(self, event: str, data: Any) -> None: ...

class Subject:
    def attach(self, obs: Observer) -> None: ...
    def detach(self, obs: Observer) -> None: ...
    def notify(self, event: str, data: Any) -> None: ...

class MergeSortExecutor(Subject):
    def sort(self, arr: List[int]) -> List[int]: ...

class QuickSortExecutor(Subject):
    def sort(self, arr: List[int]) -> List[int]: ...

class BFSExecutor(Subject):
    def bfs(self, graph: Dict[Any, List[Any]], start: Any) -> Set[Any]: ...
```

**Class & Data Structure Definitions**

* `Observer` (abstract base or `Protocol`): must implement `update(event, data)`.
* `Subject`: holds `self._observers: List[Observer]`.
* Executors inherit from `Subject` and call `self.notify(...)` at each instrumentation point.

**Edge Case Behavior**

* **Empty inputs**: MergeSort returns `[]`, QuickSort returns `[]` without error.
* **Duplicate values**: both sorts preserve stability where possible.
* **Reversed input**: QuickSort still partitions correctly.
* **Missing BFS start**: emits an `'error'` event and returns an empty set.
* **Cycle in BFS**: visited set prevents infinite loops.
* **Observer error**: caught and printed or logged (subject continues).
* **One-shot detach**: happens immediately after the triggering event.

**Constraints**

* Use only Python standard library.
* No global state; all state lives inside executor and observer instances.
* No blocking or I/O inside core algorithm methods (prints only via observers).
* Maintain algorithmic complexity as specified.

**Important Notes**

* Everything must be in one file and we need full and complete implementation.
* Adding new observers (e.g. metrics collectors, visualizers) requires no changes to the executors.
* Observers may be composed, filtered, and dynamically removed at runtime.


In [ ]:
# main

"""
Implements the Observer Pattern and three algorithm executors
(MergeSort, QuickSort, BFS) that emit events to observers.
"""

from abc import ABC, abstractmethod
from collections import deque
from typing import Any, Dict, List, Set


class Observer(ABC):
    """
    Abstract Observer requiring an update(event, data) method.
    """

    @abstractmethod
    def update(self, event: str, data: Any) -> None:
        pass


class Subject:
    """
    Maintains a list of observers and notifies them of events.
    """

    def __init__(self):
        self._observers: List[Observer] = []

    def attach(self, observer: Observer) -> None:
        if observer not in self._observers:
            self._observers.append(observer)

    def detach(self, observer: Observer) -> None:
        if observer in self._observers:
            self._observers.remove(observer)

    def notify(self, event: str, data: Any) -> None:
        for observer in list(self._observers):
            try:
                observer.update(event, data)
            except Exception as error:
                print(f"[error in observer] {error}")


class MergeSortExecutor(Subject):
    """
    MergeSort executor emitting start/compare/merge/end events.
    """

    def sort(self, arr: List[int]) -> List[int]:
        self.notify("start", arr.copy())
        if len(set(arr)) <= 1:
            self.notify("end", arr.copy())
            return arr.copy()
        result = self._merge_sort(arr)
        self.notify("end", result.copy())
        return result

    def _merge_sort(self, arr: List[int]) -> List[int]:
        if len(arr) <= 1:
            return arr
        mid = len(arr) // 2
        left = self._merge_sort(arr[:mid])
        right = self._merge_sort(arr[mid:])
        merged = self._merge(left, right)
        self.notify("merge", merged.copy())
        return merged

    def _merge(self, left: List[int], right: List[int]) -> List[int]:
        i = j = 0
        merged: List[int] = []
        while i < len(left) and j < len(right):
            self.notify("compare", {"left": left[i], "right": right[j]})
            if left[i] < right[j]:
                merged.append(left[i])
                i += 1
            else:
                merged.append(right[j])
                j += 1
        merged.extend(left[i:])
        merged.extend(right[j:])
        return merged


class QuickSortExecutor(Subject):
    """
    QuickSort executor emitting start/pivot/compare/swap/end events.
    """

    def sort(self, arr: List[int]) -> List[int]:
        self.notify("start", arr.copy())
        self._quick_sort(arr, 0, len(arr) - 1)
        self.notify("end", arr.copy())
        return arr

    def _quick_sort(self, arr: List[int], low: int, high: int) -> None:
        if low < high:
            p = self._partition(arr, low, high)
            self._quick_sort(arr, low, p - 1)
            self._quick_sort(arr, p + 1, high)

    def _partition(self, arr: List[int], low: int, high: int) -> int:
        pivot = arr[high]
        self.notify("pivot", pivot)
        i = low
        for j in range(low, high):
            self.notify("compare", {"i": j, "val": arr[j], "pivot": pivot})
            if arr[j] < pivot:
                arr[i], arr[j] = arr[j], arr[i]
                self.notify("swap", {"i": i, "j": j, "arr": arr.copy()})
                i += 1
        arr[i], arr[high] = arr[high], arr[i]
        self.notify("swap", {"i": i, "j": high, "arr": arr.copy()})
        return i


class BFSExecutor(Subject):
    """
    BFS executor emitting bfs_start/visit/enqueue/bfs_end or error.
    """

    def bfs(self, graph: Dict[Any, List[Any]], start: Any) -> Set[Any]:
        if start not in graph:
            self.notify("error", f"start node {start!r} not in graph")
            return set()

        visited: Set[Any] = set()
        queue = deque([start])
        self.notify("bfs_start", start)

        while queue:
            node = queue.popleft()
            if node not in visited:
                visited.add(node)
                self.notify("visit", node)
                for neighbor in graph.get(node, []):
                    self.notify("enqueue", neighbor)
                    if neighbor not in visited:
                        queue.append(neighbor)

        self.notify("bfs_end", visited.copy())
        return visited


class PrintObserver(Observer):
    """
    Observer that prints every event.
    """

    def update(self, event: str, data: Any) -> None:
        print(f"[{event}] {data}")


class FilterObserver(Observer):
    """
    Observer that only logs a subset of event types.
    """

    def __init__(self, allowed: List[str]) -> None:
        self.allowed = set(allowed)

    def update(self, event: str, data: Any) -> None:
        if event in self.allowed:
            print(f">>> filtered {event}: {data}")


class OneShotObserver(Observer):
    """
    Unsubscribes itself after the first occurrence of its trigger.
    """

    def __init__(self, subject: Subject, trigger: str) -> None:
        self.subject = subject
        self.trigger = trigger

    def update(self, event: str, data: Any) -> None:
        if event == self.trigger:
            self.subject.detach(self)


class ErrorObserver(Observer):
    """
    Raises an error when a specified event occurs.
    """

    def __init__(self, fail_on: str) -> None:
        self.fail_on = fail_on

    def update(self, event: str, data: Any) -> None:
        if event == self.fail_on:
            raise RuntimeError(f"intentional failure at {event}")
        print(f"ErrorObserver saw {event}")


In [ ]:
# test

"""
test.py

Unit tests for Subject/Observer framework and executors per prompt.
"""

import unittest
from collections import deque
from typing import Any, List, Dict, Set

import main
from main import (
    Subject,
    MergeSortExecutor,
    QuickSortExecutor,
    BFSExecutor,
    FilterObserver,
    OneShotObserver,
    ErrorObserver,
)


class RecordingObserver:
    """Records received (event, data) tuples."""
    def __init__(self) -> None:
        self.events: List[tuple[str, Any]] = []

    def update(self, event: str, data: Any) -> None:
        self.events.append((event, data))


class TestObserverFramework(unittest.TestCase):
    """Comprehensive tests covering every requirement."""

    def test_subject_attach_detach_notify(self) -> None:
        subj = Subject()
        rec = RecordingObserver()

        subj.detach(rec)
        subj.notify("foo", 1)
        self.assertEqual(rec.events, [])

        subj.attach(rec)
        subj.notify("bar", "x")
        self.assertEqual(rec.events, [("bar", "x")])

        rec.events.clear()
        subj.detach(rec)
        subj.notify("baz", None)
        self.assertEqual(rec.events, [])

    def test_merge_sort_basic_and_events(self) -> None:
        ms = MergeSortExecutor()
        rec = RecordingObserver()
        ms.attach(rec)

        sorted_arr = ms.sort([3, 1, 2])
        self.assertEqual(sorted_arr, [1, 2, 3])

        names = [evt for evt, _ in rec.events]
        self.assertEqual(names[0], "start")
        self.assertEqual(names[-1], "end")
        self.assertIn("compare", names)
        self.assertIn("merge", names)

    def test_merge_sort_empty_and_duplicates(self) -> None:
        ms = MergeSortExecutor()
        rec = RecordingObserver()
        ms.attach(rec)

        self.assertEqual(ms.sort([]), [])
        names = [evt for evt, _ in rec.events]
        self.assertEqual(names, ["start", "end"])

        rec.events.clear()
        self.assertEqual(ms.sort([2, 2, 2]), [2, 2, 2])
        names = [evt for evt, _ in rec.events]
        self.assertEqual(names, ["start", "end"])

    def test_quick_sort_basic_and_events(self) -> None:
        qs = QuickSortExecutor()
        rec = RecordingObserver()
        qs.attach(rec)

        result = qs.sort([5, 4, 3, 2, 1])
        self.assertEqual(result, [1, 2, 3, 4, 5])

        names = [evt for evt, _ in rec.events]
        self.assertEqual(names[0], "start")
        self.assertEqual(names[-1], "end")
        self.assertTrue(any(e == "pivot" for e in names))
        self.assertTrue(any(e == "compare" for e in names))
        self.assertTrue(any(e == "swap" for e in names))

    def test_quick_sort_identical_elements(self) -> None:
        qs = QuickSortExecutor()
        rec = RecordingObserver()
        qs.attach(rec)

        self.assertEqual(qs.sort([1, 1, 1]), [1, 1, 1])
        names = [evt for evt, _ in rec.events]
        self.assertTrue(any(e == "pivot" for e in names))

    def test_bfs_missing_start_only_error(self) -> None:
        bfs = BFSExecutor()
        rec = RecordingObserver()
        bfs.attach(rec)

        result = bfs.bfs({"A": ["B"]}, "Z")
        self.assertEqual(result, set())
        self.assertEqual(
            rec.events,
            [("error", "start node 'Z' not in graph")]
        )

    def test_bfs_cycle_and_event_order(self) -> None:
        graph: Dict[str, List[str]] = {
            "A": ["B", "C"],
            "B": ["A", "D", "E"],
            "C": ["A", "F"],
            "D": [],
            "E": ["F"],
            "F": []
        }
        bfs = BFSExecutor()
        rec = RecordingObserver()
        bfs.attach(rec)

        visited = bfs.bfs(graph, "A")
        self.assertEqual(visited, {"A", "B", "C", "D", "E", "F"})

        names = [evt for evt, _ in rec.events]
        self.assertEqual(names[0], "bfs_start")
        self.assertEqual(names[-1], "bfs_end")
        self.assertIn("visit", names)
        self.assertIn("enqueue", names)

    def test_bfs_uses_deque_for_performance(self) -> None:
        fn_globals = BFSExecutor.bfs.__globals__
        from collections import deque as _dq  # noqa: F401
        self.assertIs(fn_globals.get("deque"), _dq)

    def test_filter_observer_no_exceptions(self) -> None:
        ms = MergeSortExecutor()
        rec = RecordingObserver()
        filt = FilterObserver(allowed=["merge"])

        ms.attach(rec)
        ms.attach(filt)
        ms.sort([3, 2, 1])  # no exceptions

        names = [evt for evt, _ in rec.events]
        self.assertIn("merge", names)

    def test_one_shot_observer_detaches(self) -> None:
        ms = MergeSortExecutor()
        one = OneShotObserver(subject=ms, trigger="merge")
        rec = RecordingObserver()
        ms.attach(rec)
        ms.attach(one)

        ms.sort([2, 1])
        before = len(ms._observers)
        ms.sort([3, 2])
        after = len(ms._observers)

        self.assertEqual(after, before)

    def test_error_observer_swallowed(self) -> None:
        ms = MergeSortExecutor()
        rec = RecordingObserver()
        err = ErrorObserver(fail_on="merge")

        ms.attach(rec)
        ms.attach(err)
        self.assertEqual(ms.sort([3, 1, 2]), [1, 2, 3])

        names = [evt for evt, _ in rec.events]
        self.assertIn("merge", names)


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/0c727f83-3990-4da0-ab02-358b8bd1a332?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F45250%2Fview

#### Model code:

```python
from typing import Protocol, Any, List, Dict, Set, Optional

# Observer Protocol
class Observer(Protocol):
    def update(self, event: str, data: Any) -> None:
        ...


# Subject Base Class
class Subject:
    def __init__(self):
        self._observers: List[Observer] = []

    def attach(self, obs: Observer) -> None:
        if obs not in self._observers:
            self._observers.append(obs)

    def detach(self, obs: Observer) -> None:
        if obs in self._observers:
            self._observers.remove(obs)

    def notify(self, event: str, data: Any) -> None:
        for observer in self._observers[:]:  # Iterate over a shallow copy to allow modification
            try:
                observer.update(event, data)
            except Exception as e:
                print(f"Observer error on event '{event}': {e}")


# MergeSort Executor
class MergeSortExecutor(Subject):
    def sort(self, arr: List[int]) -> List[int]:
        self.notify('start', arr)
        sorted_arr = self._merge_sort(arr)
        self.notify('end', sorted_arr)
        return sorted_arr

    def _merge_sort(self, arr: List[int]) -> List[int]:
        if len(arr) <= 1:
            return arr

        mid = len(arr) // 2
        left = self._merge_sort(arr[:mid])
        right = self._merge_sort(arr[mid:])

        return self._merge(left, right)

    def _merge(self, left: List[int], right: List[int]) -> List[int]:
        merged = []
        i = j = 0

        while i < len(left) and j < len(right):
            self.notify('compare', (left[i], right[j]))
            if left[i] <= right[j]:
                merged.append(left[i])
                i += 1
            else:
                merged.append(right[j])
                j += 1

        merged.extend(left[i:])
        merged.extend(right[j:])
        self.notify('merge', merged)
        return merged


# QuickSort Executor
class QuickSortExecutor(Subject):
    def sort(self, arr: List[int]) -> List[int]:
        self.notify('start', arr)
        self._quick_sort(arr, 0, len(arr) - 1)
        self.notify('end', arr)
        return arr

    def _quick_sort(self, arr: List[int], low: int, high: int) -> None:
        if low < high:
            pivot_index = self._partition(arr, low, high)
            self._quick_sort(arr, low, pivot_index - 1)
            self._quick_sort(arr, pivot_index + 1, high)

    def _partition(self, arr: List[int], low: int, high: int) -> int:
        pivot = arr[high]
        self.notify('pivot', pivot)
        i = low

        for j in range(low, high):
            self.notify('compare', (arr[j], pivot))
            if arr[j] <= pivot:
                arr[i], arr[j] = arr[j], arr[i]
                self.notify('swap', (arr[i], arr[j]))
                i += 1

        arr[i], arr[high] = arr[high], arr[i]
        self.notify('swap', (arr[i], arr[high]))
        return i


# BFS Executor
class BFSExecutor(Subject):
    def bfs(self, graph: Dict[Any, List[Any]], start: Optional[Any]) -> Set[Any]:
        if start not in graph:
            self.notify('error', f"Start node '{start}' not in graph")  # <Issue>: Spec expects a 'bfs_start' event before any 'error' notification
            return set()

        visited = set()
        queue = [start]
        self.notify('bfs_start', start)

        while queue:
            node = queue.pop(0)  # <Issue>: list.pop(0) is O(n), making BFS O(n^2) instead of the required O(V+E)
            if node not in visited:
                self.notify('visit', node)
                visited.add(node)
                for neighbor in graph.get(node, []):
                    if neighbor not in visited:
                        queue.append(neighbor)
                        self.notify('enqueue', neighbor)

        self.notify('bfs_end', visited)
        return visited


# Observer Implementations

class PrintObserver:
    def update(self, event: str, data: Any) -> None:
        print(f"PrintObserver: {event} - {data}")


class FilterObserver:
    def __init__(self, *events_to_filter: str):
        self.events_to_filter = set(events_to_filter)

    def update(self, event: str, data: Any) -> None:
        if event in self.events_to_filter:
            print(f"FilterObserver: {event} - {data}")


class OneShotObserver:
    def __init__(self, trigger_event: str, subject: Subject):
        self.trigger_event = trigger_event
        self.subject = subject

    def update(self, event: str, data: Any) -> None:
        print(f"OneShotObserver: {event} - {data}")
        if event == self.trigger_event:
            self.subject.detach(self)
            print(f"OneShotObserver detached on trigger event: {event}")


class ErrorObserver:
    def __init__(self, error_event: str):
        self.error_event = error_event

    def update(self, event: str, data: Any) -> None:
        if event == self.error_event:
            raise RuntimeError(f"ErrorObserver triggered on event: {event}")


# Example Usage
if __name__ == "__main__":
    # Test MergeSortExecutor
    m_sort = MergeSortExecutor()
    print_obs = PrintObserver()
    m_sort.attach(print_obs)
    m_sort.attach(OneShotObserver('merge', m_sort))
    m_sort.attach(FilterObserver('start', 'end'))
    m_sort.attach(ErrorObserver('merge'))
    print(m_sort.sort([3, 2, 1, 4, 5]))

    # Test QuickSortExecutor
    q_sort = QuickSortExecutor()
    q_sort.attach(print_obs)
    q_sort.sort([10, 7, 9, 1, 5])

    # Test BFSExecutor
    bfs_exec = BFSExecutor()
    graph = {
        'A': ['B', 'C'],
        'B': ['A', 'D', 'E'],
        'C': ['A', 'F'],
        'D': [],
        'E': ['F'],
        'F': []
    }
    bfs_exec.attach(print_obs)
    bfs_exec.bfs(graph, 'A')
    bfs_exec.bfs(graph, 'Z')  # Should emit an error event

```